#Librerias importadas y funcionalidades necesarias

In [438]:
import numpy as np
import pandas as pd
import json
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

import nltk
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import wordnet

Se importan las librerias y las funcionalidades de lemmatizer

In [439]:
nltk.download("wordnet")
nltk.download("punkt")
nltk.download("averaged_perceptron_tagger")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

Se descargan las funcionalidades de wordnet, punkt y el perceptron categorizador de promedios

#Descarga de datos preprocesados

In [440]:
!wget -nc http://atit.upslp.edu.mx/~omar/nlp/tmdb_5000_movies.csv

File ‘tmdb_5000_movies.csv’ already there; not retrieving.



Se descarga el archivo de datos de 5000 peliculas

In [441]:
df = pd.read_csv('tmdb_5000_movies.csv')
df

budget                                             genres  \
0     237000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
1     300000000  [{"id": 12, "name": "Adventure"}, {"id": 14, "...   
2     245000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
3     250000000  [{"id": 28, "name": "Action"}, {"id": 80, "nam...   
4     260000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
...         ...                                                ...   
4798     220000  [{"id": 28, "name": "Action"}, {"id": 80, "nam...   
4799       9000  [{"id": 35, "name": "Comedy"}, {"id": 10749, "...   
4800          0  [{"id": 35, "name": "Comedy"}, {"id": 18, "nam...   
4801          0                                                 []   
4802          0                [{"id": 99, "name": "Documentary"}]   

                                               homepage      id  \
0                           http://www.avatarmovie.com/   19995   
1          http://disney.go.com/disneypictures/pirates/     285   
2           http://www.sonypictures.com/movies/spectre/  206647   
3                    http://www.thedarkknightrises.com/   49026   
4                  http://movies.disney.com/john-carter   49529   
...                                                 ...     ...   
4798                                                NaN    9367   
4799                                                NaN   72766   
4800  http://www.hallmarkchannel.com/signedsealeddel...  231617   
4801                        http://shanghaicalling.com/  126186   
4802                                                NaN   25975   

                                               keywords original_language  \
0     [{"id": 1463, "name": "culture clash"}, {"id":...                en   
1     [{"id": 270, "name": "ocean"}, {"id": 726, "na...                en   
2     [{"id": 470, "name": "spy"}, {"id": 818, "name...                en   
3     [{"id": 849, "name": "dc comics"}, {"id": 853,...                en   
4     [{"id": 818, "name": "based on novel"}, {"id":...                en   
...                                                 ...               ...   
4798  [{"id": 5616, "name": "united states\u2013mexi...                es   
4799                                                 []                en   
4800  [{"id": 248, "name": "date"}, {"id": 699, "nam...                en   
4801                                                 []                en   
4802  [{"id": 1523, "name": "obsession"}, {"id": 224...                en   

                                original_title  \
0                                       Avatar   
1     Pirates of the Caribbean: At World's End   
2                                      Spectre   
3                        The Dark Knight Rises   
4                                  John Carter   
...                                        ...   
4798                               El Mariachi   
4799                                 Newlyweds   
4800                 Signed, Sealed, Delivered   
4801                          Shanghai Calling   
4802                         My Date with Drew   

                                               overview  popularity  \
0     In the 22nd century, a paraplegic Marine is di...  150.437577   
1     Captain Barbossa, long believed to be dead, ha...  139.082615   
2     A cryptic message from Bond’s past sends him o...  107.376788   
3     Following the death of District Attorney Harve...  112.312950   
4     John Carter is a war-weary, former military ca...   43.926995   
...                                                 ...         ...   
4798  El Mariachi just wants to play his guitar and ...   14.269792   
4799  A newlywed couple's honeymoon is upended by th...    0.642552   
4800  "Signed, Sealed, Delivered" introduces a dedic...    1.444476   
4801  When ambitious New York attorney Sam is sent t...    0.857008   
4802  Ever since the second grade when he first saw ... 

Se asigna el archivo y se convierte en un dataframe

In [442]:
df.head()

budget                                             genres  \
0  237000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
1  300000000  [{"id": 12, "name": "Adventure"}, {"id": 14, "...   
2  245000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
3  250000000  [{"id": 28, "name": "Action"}, {"id": 80, "nam...   
4  260000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   

                                       homepage      id  \
0                   http://www.avatarmovie.com/   19995   
1  http://disney.go.com/disneypictures/pirates/     285   
2   http://www.sonypictures.com/movies/spectre/  206647   
3            http://www.thedarkknightrises.com/   49026   
4          http://movies.disney.com/john-carter   49529   

                                            keywords original_language  \
0  [{"id": 1463, "name": "culture clash"}, {"id":...                en   
1  [{"id": 270, "name": "ocean"}, {"id": 726, "na...                en   
2  [{"id": 470, "name": "spy"}, {"id": 818, "name...                en   
3  [{"id": 849, "name": "dc comics"}, {"id": 853,...                en   
4  [{"id": 818, "name": "based on novel"}, {"id":...                en   

                             original_title  \
0                                    Avatar   
1  Pirates of the Caribbean: At World's End   
2                                   Spectre   
3                     The Dark Knight Rises   
4                               John Carter   

                                            overview  popularity  \
0  In the 22nd century, a paraplegic Marine is di...  150.437577   
1  Captain Barbossa, long believed to be dead, ha...  139.082615   
2  A cryptic message from Bond’s past sends him o...  107.376788   
3  Following the death of District Attorney Harve...  112.312950   
4  John Carter is a war-weary, former military ca...   43.926995   

                                production_companies  \
0  [{"name": "Ingenious Film Partners", "id": 289...   
1  [{"name": "Walt Disney Pictures", "id": 2}, {"...   
2  [{"name": "Columbia Pictures", "id": 5}, {"nam...   
3  [{"name": "Legendary Pictures", "id": 923}, {"...   
4        [{"name": "Walt Disney Pictures", "id": 2}]   

                                production_countries release_date     revenue  \
0  [{"iso_3166_1": "US", "name": "United States o...   2009-12-10  2787965087   
1  [{"iso_3166_1": "US", "name": "United States o...   2007-05-19   961000000   
2  [{"iso_3166_1": "GB", "name": "United Kingdom"...   2015-10-26   880674609   
3  [{"iso_3166_1": "US", "name": "United States o...   2012-07-16  1084939099   
4  [{"iso_3166_1": "US", "name": "United States o...   2012-03-07   284139100   

   runtime                                   spoken_languages    status  \
0    162.0  [{"iso_639_1": "en", "name": "English"}, {"iso...  Released   
1    169.0           [{"iso_639_1": "en", "name": "English"}]  Released   
2    148.0  [{"iso_639_1": "fr", "name": "Fran\u00e7ais"},...  Released   
3    165.0           [{"iso_639_1": "en", "name": "English"}]  Released   
4    132.0           [{"iso_639_1": "en", "name": "English"}]  Released   

                                          tagline  \
0                     Enter the World of Pandora.   
1  At the end of the world, the adventure begins.   
2                           A Plan No One Escapes   
3                                 The Legend Ends   
4            Lost in our world, found in another.   

                                      title  vote_average  vote_count  
0                                    Avatar           7.2       11800  
1  Pirates of the Caribbean: At World's End           6.9        4500  
2                                   Spectre           6.3        4466  
3                     The Dark Knight Rises           7.6        9106  
4                               John Carter           6.1        2124

#Seleccion de palabras utiles para el dataframe y filtrado de formato

In [443]:
dfgenres = df.drop(['budget',	'homepage','id','keywords',	'original_language',	'original_title',	'overview',	'popularity',	'production_companies',	'production_countries',	'release_date',	'revenue',	'runtime',	'spoken_languages',	'status',	'tagline',	'title',	'vote_average',	'vote_count'], axis=1)  # axis=1 specifies that we're removing columns

dfgenres

genres
0     [{"id": 28, "name": "Action"}, {"id": 12, "nam...
1     [{"id": 12, "name": "Adventure"}, {"id": 14, "...
2     [{"id": 28, "name": "Action"}, {"id": 12, "nam...
3     [{"id": 28, "name": "Action"}, {"id": 80, "nam...
4     [{"id": 28, "name": "Action"}, {"id": 12, "nam...
...                                                 ...
4798  [{"id": 28, "name": "Action"}, {"id": 80, "nam...
4799  [{"id": 35, "name": "Comedy"}, {"id": 10749, "...
4800  [{"id": 35, "name": "Comedy"}, {"id": 18, "nam...
4801                                                 []
4802                [{"id": 99, "name": "Documentary"}]

[4803 rows x 1 columns]

Se eliminan columnas innecesarias para separar los generos del dataframe y remover el formato JSON

In [444]:
dfgenres = pd.DataFrame(dfgenres)

# Paso 1: Extrae las strings JSON-like de la columna 'genres'
json_strings = dfgenres['genres']

# Paso 2: Convertir cada JSON-like string a una lista de diccionario en python
parsed_data = [json.loads(json_string) for json_string in json_strings]

# 'parsed_data' contiene una lista de listas de diccionarios por cada fila

# Paso 3: Realizar operaciones necesarias en la informacion extraida
names_list =[]
for item_list in parsed_data:
    names = [item['name'] for item in item_list]
    names_list.append(names)


# Paso 4: Añadir los datos modificados devuelta al dataframe
dfgenres = pd.DataFrame({'parsed_names': names_list})


dfgenres

parsed_names
0     [Action, Adventure, Fantasy, Science Fiction]
1                      [Adventure, Fantasy, Action]
2                        [Action, Adventure, Crime]
3                  [Action, Crime, Drama, Thriller]
4              [Action, Adventure, Science Fiction]
...                                             ...
4798                      [Action, Crime, Thriller]
4799                              [Comedy, Romance]
4800             [Comedy, Drama, Romance, TV Movie]
4801                                             []
4802                                  [Documentary]

[4803 rows x 1 columns]

Una vez eliminado del formato JSON, se procede a limpiar los caracteres adicionales

In [445]:
# Convertir el Dataframe a un array tipo Numpy y unir los valores de cada fila
dfgenres = dfgenres.to_numpy()
dfgenres = [' '.join(map(str, row)) for row in dfgenres]

#Crear un nuevo dataframe con las strings transformadas
dfgenres = pd.DataFrame({'genres': dfgenres})

# mostrar el nuevo DataFrame
dfgenres

genres
0     ['Action', 'Adventure', 'Fantasy', 'Science Fi...
1                    ['Adventure', 'Fantasy', 'Action']
2                      ['Action', 'Adventure', 'Crime']
3              ['Action', 'Crime', 'Drama', 'Thriller']
4            ['Action', 'Adventure', 'Science Fiction']
...                                                 ...
4798                    ['Action', 'Crime', 'Thriller']
4799                              ['Comedy', 'Romance']
4800         ['Comedy', 'Drama', 'Romance', 'TV Movie']
4801                                                 []
4802                                    ['Documentary']

[4803 rows x 1 columns]

Se transforma el contenido del dataframe en strings

In [446]:

# Remover el primer y ultimo caracter de cada fila, asi como , '' y ', siendo remplazados por ''
dfgenres['genres'] = dfgenres['genres'].str[1:]
dfgenres['genres'] = dfgenres['genres'].str[:-1]
dfgenres['genres'] = dfgenres['genres'].str.replace(',', '').str.replace('', '').str.replace("'", '')

# mostrar el nuevo dataframe
dfgenres


genres
0     Action Adventure Fantasy Science Fiction
1                     Adventure Fantasy Action
2                       Action Adventure Crime
3                  Action Crime Drama Thriller
4             Action Adventure Science Fiction
...                                        ...
4798                     Action Crime Thriller
4799                            Comedy Romance
4800             Comedy Drama Romance TV Movie
4801                                          
4802                               Documentary

[4803 rows x 1 columns]

Se eliminan los caracteres adicionales de sobra del string

In [447]:
dfkeywords = df.drop(['budget',	'homepage','id','genres',	'original_language',	'original_title',	'overview',	'popularity',	'production_companies',	'production_countries',	'release_date',	'revenue',	'runtime',	'spoken_languages',	'status',	'tagline',	'title',	'vote_average',	'vote_count'], axis=1)  # axis=1 specifies that we're removing columns
dfkeywords

keywords
0     [{"id": 1463, "name": "culture clash"}, {"id":...
1     [{"id": 270, "name": "ocean"}, {"id": 726, "na...
2     [{"id": 470, "name": "spy"}, {"id": 818, "name...
3     [{"id": 849, "name": "dc comics"}, {"id": 853,...
4     [{"id": 818, "name": "based on novel"}, {"id":...
...                                                 ...
4798  [{"id": 5616, "name": "united states\u2013mexi...
4799                                                 []
4800  [{"id": 248, "name": "date"}, {"id": 699, "nam...
4801                                                 []
4802  [{"id": 1523, "name": "obsession"}, {"id": 224...

[4803 rows x 1 columns]

Se realiza el mismo procedimiento al que se le hizo a la categoria de generos  con las keywords, ya que tambien estan en formato JSON

In [448]:
dfkeywords = pd.DataFrame(dfkeywords)

# Paso 1: Extrae las strings JSON-like de la columna 'keywords'
json_strings = dfkeywords['keywords']

# Paso 2: Convertir cada JSON-like string a una lista de diccionario en python
parsed_data = [json.loads(json_string) for json_string in json_strings]

# 'parsed_data' contiene una lista de listas de diccionarios por cada fila

# Paso 3: Realizar operaciones necesarias en la informacion extraida
names_list =[]
for item_list in parsed_data:
    names = [item['name'] for item in item_list]
    names_list.append(names)



# Paso 4: Añadir los datos modificados devuelta al dataframe
dfkeywords = pd.DataFrame({'parsed_names': names_list})


dfkeywords

parsed_names
0     [culture clash, future, space war, space colon...
1     [ocean, drug abuse, exotic island, east india ...
2     [spy, based on novel, secret agent, sequel, mi...
3     [dc comics, crime fighter, terrorist, secret i...
4     [based on novel, mars, medallion, space travel...
...                                                 ...
4798  [united states–mexico barrier, legs, arms, pap...
4799                                                 []
4800  [date, love at first sight, narration, investi...
4801                                                 []
4802          [obsession, camcorder, crush, dream girl]

[4803 rows x 1 columns]

In [449]:
# Convertir el Dataframe a un array tipo Numpy y unir los valores de cada fila
dfkeywords = dfkeywords.to_numpy()
dfkeywords = [' '.join(map(str, row)) for row in dfkeywords]

#Crear un nuevo dataframe con las strings transformadas
dfkeywords = pd.DataFrame({'keywords': dfkeywords})

# mostrar el nuevo dataframe
dfkeywords

keywords
0     ['culture clash', 'future', 'space war', 'spac...
1     ['ocean', 'drug abuse', 'exotic island', 'east...
2     ['spy', 'based on novel', 'secret agent', 'seq...
3     ['dc comics', 'crime fighter', 'terrorist', 's...
4     ['based on novel', 'mars', 'medallion', 'space...
...                                                 ...
4798  ['united states–mexico barrier', 'legs', 'arms...
4799                                                 []
4800  ['date', 'love at first sight', 'narration', '...
4801                                                 []
4802  ['obsession', 'camcorder', 'crush', 'dream girl']

[4803 rows x 1 columns]

In [450]:

# Remover el primer y ultimo caracter de cada fila, asi como , '' y ', siendo remplazados por ''
dfkeywords['keywords'] = dfkeywords['keywords'].str[1:]
dfkeywords['keywords'] = dfkeywords['keywords'].str[:-1]
dfkeywords['keywords'] = dfkeywords['keywords'].str.replace(',', '').str.replace('', '').str.replace("'", '')

# morstrar el nuevo dataframe
dfkeywords


keywords
0     culture clash future space war space colony so...
1     ocean drug abuse exotic island east india trad...
2     spy based on novel secret agent sequel mi6 bri...
3     dc comics crime fighter terrorist secret ident...
4     based on novel mars medallion space travel pri...
...                                                 ...
4798  united states–mexico barrier legs arms paper k...
4799                                                   
4800  date love at first sight narration investigati...
4801                                                   
4802               obsession camcorder crush dream girl

[4803 rows x 1 columns]

Una vez eliminados elementos no necesarios de la columan Keywords visualizamos nuestros resultados

In [451]:
genrekeyword_df = pd.concat([dfgenres, dfkeywords], axis=1)
genrekeyword_df = genrekeyword_df[['genres','keywords']]

genrekeyword_df = pd.concat([genrekeyword_df , df.drop(['budget',	'homepage','id','genres',	'original_language',	'original_title',	'title',	'popularity',	'production_companies',	'production_countries',	'release_date',	'revenue',	'runtime',	'spoken_languages',	'status',	'tagline',	'keywords',	'vote_average',	'vote_count'], axis=1)], axis=1)

genrekeyword_df['overview'] = genrekeyword_df['overview'].str.replace(',', '').str.replace('', '').str.replace("'", '')
genrekeyword_df

genres  \
0     Action Adventure Fantasy Science Fiction   
1                     Adventure Fantasy Action   
2                       Action Adventure Crime   
3                  Action Crime Drama Thriller   
4             Action Adventure Science Fiction   
...                                        ...   
4798                     Action Crime Thriller   
4799                            Comedy Romance   
4800             Comedy Drama Romance TV Movie   
4801                                             
4802                               Documentary   

                                               keywords  \
0     culture clash future space war space colony so...   
1     ocean drug abuse exotic island east india trad...   
2     spy based on novel secret agent sequel mi6 bri...   
3     dc comics crime fighter terrorist secret ident...   
4     based on novel mars medallion space travel pri...   
...                                                 ...   
4798  united states–mexico barrier legs arms paper k...   
4799                                                      
4800  date love at first sight narration investigati...   
4801                                                      
4802               obsession camcorder crush dream girl   

                                               overview  
0     In the 22nd century a paraplegic Marine is dis...  
1     Captain Barbossa long believed to be dead has ...  
2     A cryptic message from Bond’s past sends him o...  
3     Following the death of District Attorney Harve...  
4     John Carter is a war-weary former military cap...  
...                                                 ...  
4798  El Mariachi just wants to play his guitar and ...  
4799  A newlywed couples honeymoon is upended by the...  
4800  "Signed Sealed Delivered" introduces a dedicat...  
4801  When ambitious New York attorney Sam is sent t...  
4802  Ever since the second grade when he first saw ...  

[4803 rows x 3 columns]

Estas son las columnas de la informacion relevante e importante de la pelicula, la cual consiste de los generos, palabras clave, y el sinopsis de la pelicula

In [452]:
# Convertir el Dataframe a un array tipo Numpy y unir los valores de cada fila
genrekeyword_df = genrekeyword_df.to_numpy()
genrekeyword_df = [' '.join(map(str, row)) for row in genrekeyword_df]

#Crear un nuevo dataframe con las strings transformadas
genrekeyword_df = pd.DataFrame({'genrekeyword': genrekeyword_df})

# morstrar el nuevo dataframe
genrekeyword_df

genrekeyword
0     Action Adventure Fantasy Science Fiction cultu...
1     Adventure Fantasy Action ocean drug abuse exot...
2     Action Adventure Crime spy based on novel secr...
3     Action Crime Drama Thriller dc comics crime fi...
4     Action Adventure Science Fiction based on nove...
...                                                 ...
4798  Action Crime Thriller united states–mexico bar...
4799  Comedy Romance  A newlywed couples honeymoon i...
4800  Comedy Drama Romance TV Movie date love at fir...
4801    When ambitious New York attorney Sam is sent...
4802  Documentary obsession camcorder crush dream gi...

[4803 rows x 1 columns]

Combinamos la informacion de genero, keywords y sinopsis en una sola columna.

In [453]:
merged_df = pd.concat([df.drop(['budget',	'homepage','id','genres',	'original_language',	'original_title',	'overview',	'popularity',	'production_companies',	'production_countries',	'release_date',	'revenue',	'runtime',	'spoken_languages',	'status',	'tagline',	'keywords',	'vote_average',	'vote_count'], axis=1), genrekeyword_df], axis=1)
merged_df = merged_df[['title', 'genrekeyword']]
merged_df

title  \
0                                       Avatar   
1     Pirates of the Caribbean: At World's End   
2                                      Spectre   
3                        The Dark Knight Rises   
4                                  John Carter   
...                                        ...   
4798                               El Mariachi   
4799                                 Newlyweds   
4800                 Signed, Sealed, Delivered   
4801                          Shanghai Calling   
4802                         My Date with Drew   

                                           genrekeyword  
0     Action Adventure Fantasy Science Fiction cultu...  
1     Adventure Fantasy Action ocean drug abuse exot...  
2     Action Adventure Crime spy based on novel secr...  
3     Action Crime Drama Thriller dc comics crime fi...  
4     Action Adventure Science Fiction based on nove...  
...                                                 ...  
4798  Action Crime Thriller united states–mexico bar...  
4799  Comedy Romance  A newlywed couples honeymoon i...  
4800  Comedy Drama Romance TV Movie date love at fir...  
4801    When ambitious New York attorney Sam is sent...  
4802  Documentary obsession camcorder crush dream gi...  

[4803 rows x 2 columns]

Se añade el titulo de la obra a cada bolsa de palabras importantes de la pelicula

In [454]:
merged_df = pd.concat([merged_df , df.drop(['budget',	'homepage','id','genres',	'original_language',	'original_title',	'title',	'popularity',	'production_companies',	'production_countries',	'release_date',	'revenue',	'runtime',	'spoken_languages',	'status',	'tagline',	'keywords',	'vote_average',	'vote_count'], axis=1)], axis=1)
merged_df['genrekeywordoverview'] = merged_df['genrekeyword'] +''+ merged_df['overview']
merged_df = merged_df.drop(['genrekeyword','overview'], axis=1)  # axis=1 specifies that we're removing columns
merged_df = merged_df.rename(columns={'genrekeywordoverview': 'movieinfo'})
merged_df

title  \
0                                       Avatar   
1     Pirates of the Caribbean: At World's End   
2                                      Spectre   
3                        The Dark Knight Rises   
4                                  John Carter   
...                                        ...   
4798                               El Mariachi   
4799                                 Newlyweds   
4800                 Signed, Sealed, Delivered   
4801                          Shanghai Calling   
4802                         My Date with Drew   

                                              movieinfo  
0     Action Adventure Fantasy Science Fiction cultu...  
1     Adventure Fantasy Action ocean drug abuse exot...  
2     Action Adventure Crime spy based on novel secr...  
3     Action Crime Drama Thriller dc comics crime fi...  
4     Action Adventure Science Fiction based on nove...  
...                                                 ...  
4798  Action Crime Thriller united states–mexico bar...  
4799  Comedy Romance  A newlywed couples honeymoon i...  
4800  Comedy Drama Romance TV Movie date love at fir...  
4801    When ambitious New York attorney Sam is sent...  
4802  Documentary obsession camcorder crush dream gi...  

[4803 rows x 2 columns]

Se realiza limpieza extra del dataframe.

In [455]:
len(merged_df)

finaldata = merged_df[["title", "movieinfo"]]          # Columnas requeridas - title and movieinfo
finaldata = finaldata.set_index('title')    # Se establece el titulo de la pelicula como indice

finaldata.head(10)

movieinfo
title                                                                                      
Avatar                                    Action Adventure Fantasy Science Fiction cultu...
Pirates of the Caribbean: At World's End  Adventure Fantasy Action ocean drug abuse exot...
Spectre                                   Action Adventure Crime spy based on novel secr...
The Dark Knight Rises                     Action Crime Drama Thriller dc comics crime fi...
John Carter                               Action Adventure Science Fiction based on nove...
Spider-Man 3                              Fantasy Action Adventure dual identity amnesia...
Tangled                                   Animation Family hostage magic horse fairy tal...
Avengers: Age of Ultron                   Action Adventure Science Fiction marvel comic ...
Harry Potter and the Half-Blood Prince    Adventure Fantasy Family witch magic broom sch...
Batman v Superman: Dawn of Justice        Action Adventure Fantasy dc comics vigilante s...

La informacion final que se tokenizara es la columna 'movieinfo' y la pelicula a la que pertenece 'title'

In [456]:
finaldata = finaldata.astype(str)

#Revisar el tipo de objeto del dataframe
print(finaldata.dtypes)

movieinfo    object
dtype: object


Revisamos los tipo de datos del dataframe y se convierten en compatibilidad de tipo string para poder realizar la tokenizacion, ya que el tokenizador solo trabaja con tipos de datos en formato string

In [457]:
finaldata["movieinfo"][0]

'Action Adventure Fantasy Science Fiction culture clash future space war space colony society space travel futuristic romance space alien tribe alien planet cgi marine soldier battle love affair anti war power relations mind and soul 3d In the 22nd century a paraplegic Marine is dispatched to the moon Pandora on a unique mission but becomes torn between following orders and protecting an alien civilization.In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

Se estableceran algunos modelos para revisar la eficiencia

In [458]:
finaldatam1 = finaldata
finaldatam2 = finaldata
finaldatam3 = finaldata

In [459]:
finaldatam3.head()

movieinfo
title                                                                                      
Avatar                                    Action Adventure Fantasy Science Fiction cultu...
Pirates of the Caribbean: At World's End  Adventure Fantasy Action ocean drug abuse exot...
Spectre                                   Action Adventure Crime spy based on novel secr...
The Dark Knight Rises                     Action Crime Drama Thriller dc comics crime fi...
John Carter                               Action Adventure Science Fiction based on nove...

In [460]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
VERB_CODES = {'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'}

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#Modelos

##Modelo 1 sin contracciones ingles americanas-britanicas

Una vez importado el tokenizador, lematizador y las stopwords del idioma ingles, asi como tambien definir los codigos de verbos

In [461]:
def preprocess_sentencesm1(text):
  text = text.lower()
  temp_sent =[]
  words = nltk.word_tokenize(text)
  tags = nltk.pos_tag(words)
  for i, word in enumerate(words):
      if tags[i][1] in VERB_CODES:
          lemmatized = lemmatizer.lemmatize(word, 'v')
      else:
          lemmatized = lemmatizer.lemmatize(word)
      if lemmatized not in stop_words and lemmatized.isalpha():
          temp_sent.append(lemmatized)

  finalsentm1 = ' '.join(temp_sent)
  finalsentm1 = finalsentm1.replace("n't", " not")
  finalsentm1 = finalsentm1.replace("'m", " am")
  finalsentm1 = finalsentm1.replace("'s", " is")
  finalsentm1 = finalsentm1.replace("'re", " are")
  finalsentm1 = finalsentm1.replace("'ll", " will")
  finalsentm1 = finalsentm1.replace("'ve", " have")
  finalsentm1 = finalsentm1.replace("'d", " would")
  return finalsentm1

finaldatam1["movieinfo_processed"]= finaldatam1["movieinfo"].apply(preprocess_sentencesm1)
finaldatam1.head()

movieinfo  \
title                                                                                         
Avatar                                    Action Adventure Fantasy Science Fiction cultu...   
Pirates of the Caribbean: At World's End  Adventure Fantasy Action ocean drug abuse exot...   
Spectre                                   Action Adventure Crime spy based on novel secr...   
The Dark Knight Rises                     Action Crime Drama Thriller dc comics crime fi...   
John Carter                               Action Adventure Science Fiction based on nove...   

                                                                        movieinfo_processed  
title                                                                                        
Avatar                                    action adventure fantasy science fiction cultu...  
Pirates of the Caribbean: At World's End  adventure fantasy action ocean drug abuse exot...  
Spectre                                   action adventure crime spy base novel secret a...  
The Dark Knight Rises                     action crime drama thriller dc comic crime fig...  
John Carter                               action adventure science fiction base novel ma...

Dentro de la informacion de la pelicula existen diferencias entre el ingles britanico y el ingles americano, realizamos una transformacion de algunas de las contracciones mas comunes para hacer el modelo de recomendaciones ligeramente mas efieciente

Cabe mencionar que este sistema de recomendaciones solo servira eficiente mente para titulos en ingles americano

In [462]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Vectorizado de la columna 'movieinfo_processed' usando TF-IDF
tfidfvec = TfidfVectorizer()
tfidf_movieid = tfidfvec.fit_transform((finaldatam1["movieinfo_processed"]))

# Buscar similaridad de coseno entre vectores
from sklearn.metrics.pairwise import cosine_similarity
cos_sim = cosine_similarity(tfidf_movieid, tfidf_movieid)


Se aplica la funcion TFidfVectorizer a las strings que preparamos que contiene la informacion de la pelicula  y definimos una variable que calcula la similaridad del coseno entre dos vectores

In [463]:
# Guardado de indices de los datos
indicesm1 = pd.Series(finaldatam1.index)

def recommendationsm1(title, cosine_sim = cos_sim):
    peliculas_recomendar = []
    index = indicesm1[indicesm1 == title].index[0]
    puntaje_similar = pd.Series(cosine_sim[index]).sort_values(ascending = False)
    top_10 = list(puntaje_similar.iloc[1:11].index)
    for i in top_10:
        peliculas_recomendar.append(list(finaldatam1.index)[i])
    return peliculas_recomendar

Se realiza un guardado de indices de similaridad usando el nombre de la pelicula ingresado y se retornan las 10 peliculas con coseno de similaridad mas cercano

##Modelo 2 con contracciones de ingles americano-britanico

In [464]:
def preprocess_sentencesm2(text):
  text = text.lower()
  temp_sent =[]
  words = nltk.word_tokenize(text)
  tags = nltk.pos_tag(words)
  for i, word in enumerate(words):
      if tags[i][1] in VERB_CODES:
          lemmatized = lemmatizer.lemmatize(word, 'v')
      else:
          lemmatized = lemmatizer.lemmatize(word)
      if lemmatized not in stop_words and lemmatized.isalpha():
          temp_sent.append(lemmatized)

  finalsentm2 = ' '.join(temp_sent)
#  finalsent = finalsent.replace("n't", " not")
#  finalsent = finalsent.replace("'m", " am")
#  finalsent = finalsent.replace("'s", " is")
#  finalsent = finalsent.replace("'re", " are")
#  finalsent = finalsent.replace("'ll", " will")
#  finalsent = finalsent.replace("'ve", " have")
#  finalsent = finalsent.replace("'d", " would")
  return finalsentm2

finaldatam2["movieinfo_processed"]= finaldatam2["movieinfo"].apply(preprocess_sentencesm2)
finaldatam2.head()

movieinfo  \
title                                                                                         
Avatar                                    Action Adventure Fantasy Science Fiction cultu...   
Pirates of the Caribbean: At World's End  Adventure Fantasy Action ocean drug abuse exot...   
Spectre                                   Action Adventure Crime spy based on novel secr...   
The Dark Knight Rises                     Action Crime Drama Thriller dc comics crime fi...   
John Carter                               Action Adventure Science Fiction based on nove...   

                                                                        movieinfo_processed  
title                                                                                        
Avatar                                    action adventure fantasy science fiction cultu...  
Pirates of the Caribbean: At World's End  adventure fantasy action ocean drug abuse exot...  
Spectre                                   action adventure crime spy base novel secret a...  
The Dark Knight Rises                     action crime drama thriller dc comic crime fig...  
John Carter                               action adventure science fiction base novel ma...

In [465]:
# Vectorizado de la columna 'movieinfo_processed' usando TF-IDF
tfidf_movieid = tfidfvec.fit_transform((finaldatam2["movieinfo_processed"]))

# Busqueda de similaridad de coseno entre vectores
cos_sim = cosine_similarity(tfidf_movieid, tfidf_movieid)

In [466]:
# Guardado de indices de los datos

indicesm2 = pd.Series(finaldatam2.index)

def recommendationsm2(title, cosine_sim = cos_sim):
    peliculas_recomendar = []
    index = indicesm2[indicesm2 == title].index[0]
    puntaje_similar = pd.Series(cosine_sim[index]).sort_values(ascending = False)
    top_10 = list(puntaje_similar.iloc[1:11].index)
    for i in top_10:
        peliculas_recomendar.append(list(finaldatam2.index)[i])
    return peliculas_recomendar

Se realiza un guardado de indices de similaridad usando el nombre de la pelicula ingresado y se retornan las 10 peliculas con coseno de similaridad mas cercano

##Modelo 3 simple tokenizer

In [467]:
finaldatam3 = finaldatam3.drop(['movieinfo_processed'], axis=1)
finaldatam3.head()

movieinfo
title                                                                                      
Avatar                                    Action Adventure Fantasy Science Fiction cultu...
Pirates of the Caribbean: At World's End  Adventure Fantasy Action ocean drug abuse exot...
Spectre                                   Action Adventure Crime spy based on novel secr...
The Dark Knight Rises                     Action Crime Drama Thriller dc comics crime fi...
John Carter                               Action Adventure Science Fiction based on nove...

In [468]:
import re
# Definir una funcion de tokenizador simple
def simple_tokenizer(text):
    # Dividir el texto en tokens usando '' como un delimitador
    tokens = re.split(r'\s+', text)
    return tokens

# Definir una funcion para aplicar la funcion tokenizadora a una columna en el dataframe
def apply_simple_tokenizer_to_column(df, column_name):
    # Aplicar la funcion tokenizadora y en la columna
    df[column_name] = df[column_name].apply(simple_tokenizer)
    return df

In [469]:
# Aplicar el tokenizador simple a la columna 'movieinfo'
finaldatam3 = apply_simple_tokenizer_to_column(finaldatam3, 'movieinfo')
finaldatam3

movieinfo
title                                                                                      
Avatar                                    [Action, Adventure, Fantasy, Science, Fiction,...
Pirates of the Caribbean: At World's End  [Adventure, Fantasy, Action, ocean, drug, abus...
Spectre                                   [Action, Adventure, Crime, spy, based, on, nov...
The Dark Knight Rises                     [Action, Crime, Drama, Thriller, dc, comics, c...
John Carter                               [Action, Adventure, Science, Fiction, based, o...
...                                                                                     ...
El Mariachi                               [Action, Crime, Thriller, united, states–mexic...
Newlyweds                                 [Comedy, Romance, A, newlywed, couples, honeym...
Signed, Sealed, Delivered                 [Comedy, Drama, Romance, TV, Movie, date, love...
Shanghai Calling                          [, When, ambitious, New, York, attorney, Sam, ...
My Date with Drew                         [Documentary, obsession, camcorder, crush, dre...

[4803 rows x 1 columns]

Aplicar el tokenizador simple a la columna 'movieinfo'

In [470]:
finaldatam3.head()

movieinfo
title                                                                                      
Avatar                                    [Action, Adventure, Fantasy, Science, Fiction,...
Pirates of the Caribbean: At World's End  [Adventure, Fantasy, Action, ocean, drug, abus...
Spectre                                   [Action, Adventure, Crime, spy, based, on, nov...
The Dark Knight Rises                     [Action, Crime, Drama, Thriller, dc, comics, c...
John Carter                               [Action, Adventure, Science, Fiction, based, o...

In [471]:
indicesm3 = pd.Series(finaldatam3.index)

def recommendationsm3(title, cosine_sim = cos_sim):
    peliculas_recomendar = []
    index = indicesm3[indicesm3 == title].index[0]
    puntaje_similar = pd.Series(cosine_sim[index]).sort_values(ascending = False)
    top_10 = list(puntaje_similar.iloc[1:11].index)
    for i in top_10:
        peliculas_recomendar.append(list(finaldatam3.index)[i])
    return peliculas_recomendar

Se realiza un guardado de indices de similaridad usando el nombre de la pelicula ingresado y se retornan las 10 peliculas con coseno de similaridad mas cercano

#Revision de resultados

In [472]:
recommendationsm3("Ice Age")


['Ice Age: Dawn of the Dinosaurs',
 'Ice Age: The Meltdown',
 'Dinosaur',
 'Year One',
 'Ice Age: Continental Drift',
 'Journey to the Center of the Earth',
 'The Croods',
 'A Sound of Thunder',
 'Quest for Fire',
 'The Clan of the Cave Bear']

In [473]:
recommendationsm1("Ice Age")


['Ice Age: Dawn of the Dinosaurs',
 'Ice Age: The Meltdown',
 'Dinosaur',
 'Year One',
 'Ice Age: Continental Drift',
 'Journey to the Center of the Earth',
 'The Croods',
 'A Sound of Thunder',
 'Quest for Fire',
 'The Clan of the Cave Bear']

In [474]:
recommendationsm2("Ice Age")


['Ice Age: Dawn of the Dinosaurs',
 'Ice Age: The Meltdown',
 'Dinosaur',
 'Year One',
 'Ice Age: Continental Drift',
 'Journey to the Center of the Earth',
 'The Croods',
 'A Sound of Thunder',
 'Quest for Fire',
 'The Clan of the Cave Bear']

Visualizacion de las diferencias de los modelos de recomendaciones

In [475]:
recommendationsm1("Scream 3")

['Disaster Movie',
 'Jackass 3D',
 'Scary Movie 2',
 '2:13',
 'Land of the Dead',
 'Grindhouse',
 'Da Sweet Blood of Jesus',
 'Scream 4',
 'Diary of the Dead',
 'We Need to Talk About Kevin']

In [476]:
recommendationsm2("Scream 3")

['Disaster Movie',
 'Jackass 3D',
 'Scary Movie 2',
 '2:13',
 'Land of the Dead',
 'Grindhouse',
 'Da Sweet Blood of Jesus',
 'Scream 4',
 'Diary of the Dead',
 'We Need to Talk About Kevin']

Visualizacion de las diferencias de los modelos de recomendaciones

In [477]:
recommendationsm1("Scary Movie 4")


['Insidious: Chapter 2',
 'The Others',
 'Aliens in the Attic',
 'Area 51',
 'Sinister 2',
 'House on Haunted Hill',
 'Hayride',
 'E.T. the Extra-Terrestrial',
 'Jimmy Neutron: Boy Genius',
 'Scary Movie 3']

In [478]:
recommendationsm2("Scary Movie 4")


['Insidious: Chapter 2',
 'The Others',
 'Aliens in the Attic',
 'Area 51',
 'Sinister 2',
 'House on Haunted Hill',
 'Hayride',
 'E.T. the Extra-Terrestrial',
 'Jimmy Neutron: Boy Genius',
 'Scary Movie 3']

Visualizacion de las diferencias de los modelos de recomendaciones

In [479]:
recommendationsm1("Paul")


['Area 51',
 'Cop Out',
 'Battleship',
 'Close Encounters of the Third Kind',
 'Mission to Mars',
 'Anatomy',
 "The General's Daughter",
 'Cat People',
 'Mars Needs Moms',
 'Dreamcatcher']

In [480]:
recommendationsm2("Paul")


['Area 51',
 'Cop Out',
 'Battleship',
 'Close Encounters of the Third Kind',
 'Mission to Mars',
 'Anatomy',
 "The General's Daughter",
 'Cat People',
 'Mars Needs Moms',
 'Dreamcatcher']

In [481]:
recommendationsm1("El Mariachi")


['Desperado',
 'Once Upon a Time in Mexico',
 'My Big Fat Independent Movie',
 'Love and Other Catastrophes',
 'A Civil Action',
 'Salvador',
 'Blazing Saddles',
 'Faster',
 'Footloose',
 'Switchback']

#Conclusion

El uso de lematizacion es limitado en algo tan multicultural, sujeto a las visiones y experiencias culturales especificas de una produccion artistica como peliculas como para poder realizar una simple implementacion y esperar a que funcione para todas las peliculas, algunas de las limitantes son:


1.   Idioma
2.   Contracciones gramaticales
3.   Raiz de las palabras extranjeras
4.   Traducciones


Se podria realizar un sistema de recomendacion que tome en cuenta todas estas variables para hacer un sistema que funcione de manera general.